In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import time

In [ ]:
working_dir = "/Users/filippo/Developer/tesi"
working_dir = r"/Volumes/Googledrive/My Drive/tesi_magistrale/tesi/results/hSBM/highlyvariable_10tissue/"
os.chdir(working_dir)
dirs = os.listdir("data")

In [ ]:
len(dirs)

In [ ]:
df_genes = pd.read_csv("genes.txt", index_col=[0])
protein_coding = df_genes[df_genes['type_of_gene']=='protein-coding']
#mirna = df_genes[df_genes['type_of_gene']=='microRNA']
select_genes = protein_coding.index.values
#select_genes = mirna.index.values

In [ ]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
#print(len(df['gene']))
df = df.loc[select_genes,:]
df.head()

In [ ]:
#fpkm
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM]{5}.txt[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
#counts
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.htseq]{6}[\.\_counts]{7}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile
    print("Nothing found")
    print(os.listdir(directory))

In [ ]:
#fpkm-uq
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM-UQ]{0,8}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
##miRNA
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.mirbase21\.mirnas\.quantification][\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
maxacceptables = 5000
added = 0
for i,cdirectory in enumerate(dirs):
    if re.match("[manifest\.txt]",cdirectory):
        continue
    if "Icon" in cdirectory:
        continue
    cfile = getFilenameFromDir("data/%s"%cdirectory)
    #put header=0 for miRNA!
    #cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t', header=0)
    cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t')
    #cdf.drop(labels=["reads_per_million_miRNA_mapped","cross-mapped"], axis=1, inplace=True) #miRNA only
    cdf.columns = ["gene", cfile[:]]
    cdf['gene'] = [gene[:15] for gene in cdf['gene']]
    cdf.set_index('gene',inplace=True)
    try:
        df.insert(1,cdf.keys()[0][:],cdf.loc[select_genes,cdf.keys()[0]].values[:60483])
        #df.insert(0,cdf.keys()[0][:],cdf.values.T[0]) #miRNA are not protein-coding
        added=added+1
    except:
        print(sys.exc_info()[0])
    if added >= maxacceptables:
        break
print(added, i)

In [ ]:
#df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [ ]:
#df.columns=['gene', 'breast', 'kidney', 'bronchus', 'brain']

In [ ]:
df.drop('type_of_gene', inplace=True, axis=1)
df.drop('lenght', inplace=True, axis=1)
df.drop('exons', inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.columns))))

In [ ]:
df.to_csv(("%s/mainTable.csv"%working_dir), index=True)

expression per gene

In [ ]:
np.sum(df.loc[0,:].values[1:])

expression per case

In [ ]:
np.sum(df.loc[:,df.keys()[1]].values)

In [ ]:
df_files=pd.read_csv("files.dat",header=0,index_col=0)
df_files[df_files.index.isin(df.columns)].to_csv("files.dat",index=True,header=True)

## Zipf

In [ ]:
A = [df.loc[:,df.keys()[9]].values, df.loc[:,df.keys()[45]].values, df.loc[:,df.keys()[42]].values]
colors = ['red', 'blue', 'yellow']
fig = plt.figure()
x = np.arange(1, len(A[0]))
plt.ylabel("frequencyi")
plt.xlabel("RANKi")
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-\\alpha}$')
for i,a in enumerate(A):
    plt.plot(np.sort(a/np.sum(a, dtype=float))[::-1], c=colors[i], label = 'file') 
plt.legend(fontsize=16)
plt.show()

## Zipf per cancer

In [ ]:
colors = ['red', 'blue', 'yellow', 'green']
fig = plt.figure(figsize=(8,6))
x = np.logspace(0, 5)
plt.ylabel("$frequency_i$", fontsize=16)
plt.xlabel("$RANK_i$", fontsize=16)
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-1}$')
for i,k in enumerate(df.keys()[1:]):
    a=df[k]
    plt.plot(np.sort(a/np.sum(a))[::-1], c=colors[i], label = k) 
plt.legend(fontsize=16)
plt.show()

In [ ]:
fig.savefig("zipfPerCancer.pdf")

# gene table

In [ ]:
genes = open("%s/genes.txt"%working_dir, "w")
for gene in cdf['gene'].values:
    genes.write("%s\n"%gene)
genes.close()

# table as image

In [ ]:
from PIL import Image

In [ ]:
bits = np.array([df.loc[i,:].values[1:] for i in np.arange(len(df['gene']))])

In [ ]:
bits.shape

In [ ]:
img = Image.fromarray(bits, mode='1')

In [ ]:
img.save("mat.bmp")